In [4]:
import pandas as pd
import ollama
import uuid
import os
import json
from tqdm import tqdm
from neo4j import GraphDatabase
from pydantic import BaseModel, Field
from typing import List

# ================= КОНФИГУРАЦИЯ =================
# Модель для логики (извлечение сущностей)
OLLAMA_MODEL = "qwen3:4b" 
OLLAMA_EMBEDDING_MODEL = "qwen3-embedding:0.6b" 

NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j", "password")

# Схема для структурированного вывода LLM
class TickerExtraction(BaseModel):
    tickers: List[str] = Field(description="List of official S&P 500 company name found in the text (e.g. ['Apple', 'Google', 'Amazon', 'Uber'])")

class FinancialGraphRAG:
    def __init__(self, uri, auth):
        self.driver = GraphDatabase.driver(uri, auth=auth)
        self.embed_dim = None 

    def close(self):
        self.driver.close()

    def get_embedding(self, text):
        """Получает вектор через Ollama"""
        try:
            # Убираем переносы, чтобы вектор был плотнее
            clean_text = text.replace("\n", " ").strip()
            if not clean_text: return []
            
            response = ollama.embeddings(model=OLLAMA_EMBEDDING_MODEL, prompt=clean_text)
            vec = response['embedding']
            
            # Запоминаем размерность при первом вызове
            if self.embed_dim is None:
                self.embed_dim = len(vec)
            return vec
        except Exception as e:
            print(f"Embedding error: {e}")
            return []

    # ================= 1. ПОСТРОЕНИЕ ТАКСОНОМИИ И ИНДЕКСА =================
    def build_taxonomy_from_csv(self, csv_path):
        """Загружает компании из локального CSV и строит скелет графа"""
        print(f"\n🏗️  Загрузка таксономии из {csv_path}...")
        
        if not os.path.exists(csv_path):
            print("❌ Файл не найден!")
            return

        df = pd.read_csv(csv_path, sep=";").fillna("Unknown")
        data = df.to_dict('records')

        query_import = """
        UNWIND $rows AS row
        MERGE (s:Sector {name: row.Sector})
        MERGE (i:Industry {name: row.Industry})
        MERGE (i)-[:PART_OF]->(s)
        
        MERGE (c:Company {ticker: row.Ticker})
        SET c.name = row.Name,
            c.description = row.Description,
            c.market_cap = row['Market Cap'],
            c.website = row.Website,
            c.employees = row.Employees
            
        MERGE (c)-[:BELONGS_TO]->(i)
        """

        with self.driver.session() as session:
            batch_size = 500
            for i in range(0, len(data), batch_size):
                batch = data[i:i + batch_size]
                session.run(query_import, rows=batch)
                print(f"   Загружено {i + len(batch)} компаний...")
            
            # Индекс по тикеру для быстрого точного поиска
            session.run("CREATE INDEX company_ticker IF NOT EXISTS FOR (c:Company) ON (c.ticker)")
            
        print("✅ Таксономия загружена.")

    def prepare_company_vectors(self):
        """
        Создает 'Богатый вектор': Name + Ticker + Description.
        Это позволяет находить 'X Corp' по запросу 'Twitter'.
        """
        print("\n🧠 Генерация векторов (Name + Description)...")
        
        # Берем данные из базы
        query_fetch = """
        MATCH (c:Company)
        RETURN c.ticker as ticker, c.name as name, c.description as desc
        """
        
        with self.driver.session() as session:
            companies = session.run(query_fetch).data()
            
            for comp in tqdm(companies, desc="Embedding Companies"):
                # Формируем насыщенный текст
                desc_short = comp['desc'][:1000] # Берём первые 1000 символов описания
                rich_text = f"Company: {comp['name']} ({comp['ticker']}). Description: {desc_short}"
                
                vector = self.get_embedding(rich_text)
                
                if vector:
                    session.run("""
                        MATCH (c:Company {ticker: $ticker})
                        CALL db.create.setNodeVectorProperty(c, 'embedding', $vector)
                    """, ticker=comp['ticker'], vector=vector)

            # Создаем векторный индекс
            if self.embed_dim:
                # Удаляем старый, если есть
                session.run("DROP INDEX company_semantic_index IF EXISTS")
                
                print(f"Создаем индекс (Dim: {self.embed_dim})...")
                session.run(f"""
                    CREATE VECTOR INDEX company_semantic_index IF NOT EXISTS
                    FOR (c:Company) ON (c.embedding)
                    OPTIONS {{indexConfig: {{
                        `vector.dimensions`: {self.embed_dim},
                        `vector.similarity_function`: 'cosine'
                    }}}}
                """)
                # Небольшая пауза для построения индекса
                import time; time.sleep(3)

    # ================= 2. ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ ДЛЯ НОВОСТЕЙ =================
    def extract_tickers_with_llm(self, text):
        """
        Использует Ollama Structured Output для получения JSON с тикерами.
        """
        prompt = f"""
        Analyze the text. Identify S&P 500 companies mentioned.
        Write names (e.g. "Google", "iPhone maker" or "Apple").
        Return JSON with a list of company names.
        Text: "{text[:2000]}"
        """
        try:
            response = ollama.chat(
                model=OLLAMA_MODEL,
                messages=[{'role': 'user', 'content': prompt}],
                format=TickerExtraction.model_json_schema(), # Гарантирует JSON
                options={'temperature': 0}
            )
            json_resp = response['message']['content']
            data = TickerExtraction.model_validate_json(json_resp)
            return data.tickers
        except Exception as e:
            # print(f"LLM Error: {e}")
            return []

    def link_extracted_entities(self, news_id, tickers):
        """
        Дополнительно проверяет связь через векторный поиск.
        Это создает связь [:SEMANTIC_MATCH] с оценкой уверенности (score).
        """
        if not tickers: return

        # Ищем по вектору ТИКЕРА (или названия, если LLM вернула название)
        # Так как наш индекс содержит Name+Ticker+Desc, вектор "AAPL" найдет компанию Apple.
        
        cypher_semantic = """
        CALL db.index.vector.queryNodes('company_semantic_index', 1, $vec)
        YIELD node AS company, score
        WHERE score > 0.75  // Порог уверенности
        
        MATCH (n:News {id: $news_id})
        MERGE (n)-[r:SEMANTIC_MATCH]->(company)
        SET r.score = score, r.matched_term = $term
        """
        
        with self.driver.session() as session:
            for term in tickers:
                # Векторизуем то, что нашла LLM (например, "AAPL")
                term_vec = self.get_embedding(term)
                if term_vec:
                    session.run(cypher_semantic, news_id=news_id, vec=term_vec, term=term)

    # ================= 3. ВАША ФУНКЦИЯ (ИНТЕГРИРОВАННАЯ) =================
    def ingest_news(self, csv_path):
        print(f"\n📰 Обработка новостей из {csv_path} с помощью модели {OLLAMA_MODEL}...")
        
        if not os.path.exists(csv_path):
            print("❌ Файл новостей не найден.")
            return

        df = pd.read_csv(csv_path)
        df['headline'] = df['headline'].fillna('')
        df['text'] = df['text'].fillna('')

        # Запрос 1: Жесткая привязка (Explicit Link)
        # Если LLM сказала "AAPL", мы ищем узел Company {ticker: 'AAPL'} и вяжем.
        cypher_explicit = """
        MERGE (n:News {id: $id})
        SET n.headline = $headline, n.text = $text, n.date = datetime()
        WITH n
        UNWIND $tickers as ticker
        MATCH (c:Company {ticker: ticker})
        MERGE (n)-[:MENTIONS {type: 'explicit'}]->(c)
        """

        success_count = 0
        
        with self.driver.session() as session:
            for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing News"):
                
                full_text = f"{row['headline']}. {row['text']}"
                news_id = str(uuid.uuid4())

                # 1. Спрашиваем LLM
                tickers = self.extract_tickers_with_llm(full_text)
                
                # Для отладки можно раскомментировать
                print(f"Found: {tickers} in '{row['headline'][:30]}...'")
                
                if tickers:
                    # 2. Пишем в базу (Явная связь по тикеру)
                    session.run(cypher_explicit, 
                                id=news_id, 
                                headline=row['headline'], 
                                text=row['text'], 
                                tickers=tickers)
                    
                    # 3. Пишем в базу (Семантическое подтверждение)
                    # Это создаст дополнительную связь SEMANTIC_MATCH со скором
                    self.link_extracted_entities(news_id, tickers)
                    
                    success_count += 1
                else:
                    # Сохраняем новость без связей ("сирота")
                    session.run("MERGE (n:News {id: $id}) SET n.headline = $h, n.text = $t", 
                                id=news_id, h=row['headline'], t=row['text'])

        print(f"\n✅ Готово! Новостей привязано к компаниям: {success_count} из {len(df)}")

# ================= ЗАПУСК =================
if __name__ == "__main__":
    # Файл таксономии (структура: Ticker;Name;...;Description;...)
    taxonomy_file = "sp500_data.csv"
    
    # Файл новостей (структура: headline,text)
    news_file = "./data/news.csv"

    rag = FinancialGraphRAG(NEO4J_URI, NEO4J_AUTH)

    try:
        # ШАГ 1: Загрузка базы компаний (если нужно обновить)
        # rag.build_taxonomy_from_csv(taxonomy_file)
        
        # ШАГ 2: Генерация умных векторов (Обязательно, если обновили таксономию!)
        # rag.prepare_company_vectors()
        
        # ШАГ 3: Загрузка новостей (Ваша функция)
        rag.ingest_news(news_file)

    finally:
        rag.close()


📰 Обработка новостей из ./data/news.csv с помощью модели qwen3:4b...


Processing News:   0%|          | 0/250 [00:00<?, ?it/s]

Found: ['AAPL'] in 'Apple integrates Generative AI...'


Processing News:   1%|          | 2/250 [00:37<1:19:02, 19.12s/it]

Found: ['Microsoft'] in 'Microsoft reports surge in Azu...'


...

Found: ['EOG'] in 'EOG Resources deploys 'super-z...'


Processing News: 100%|█████████▉| 249/250 [2:34:21<00:31, 31.88s/it]

Found: ['EPAM'] in 'EPAM Systems diversifies deliv...'


Processing News: 100%|██████████| 250/250 [2:34:46<00:00, 37.15s/it]

Found: ['EQT'] in 'EQT Corp pauses natural gas pr...'

✅ Готово! Новостей привязано к компаниям: 227 из 250
